# CLARISSA Code Examples

Executable Python code from the [Reservoir Basics Tutorial](reservoir-basics.md).

## Pore Volume Calculation

In [ ]:
# Reservoir parameters (SPE9 model)
nx, ny, nz = 24, 25, 15  # Grid dimensions
dx, dy, dz = 300, 300, 50  # Cell sizes in feet
porosity = 0.087  # Average porosity
ntg = 1.0  # Net-to-gross ratio

# Calculate volumes
bulk_volume = nx * ny * nz * dx * dy * dz  # ft³
pore_volume = bulk_volume * porosity * ntg
pore_volume_bbl = pore_volume / 5.615  # Convert to barrels

print(f'Grid: {nx} x {ny} x {nz} = {nx*ny*nz:,} cells')
print(f'Pore volume: {pore_volume_bbl/1e6:.2f} MMbbl')

## Material Balance - OOIP

In [ ]:
def calculate_ooip(np_cum, bo, boi, delta_p, ce):
    """Calculate OOIP using simplified material balance."""
    return (np_cum * bo) / (boi * ce * delta_p)

# Example
N = calculate_ooip(
    np_cum=1_500_000,  # STB produced
    bo=1.25,
    boi=1.30,
    delta_p=500,  # psi
    ce=15e-6  # 1/psi
)

print(f'Estimated OOIP: {N/1e6:.1f} MMSTB')